In [1]:
!pip install --upgrade ibm-watson

Requirement already up-to-date: ibm-watson in c:\program files\python36\lib\site-packages (4.4.1)


In [2]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = ''

In [3]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)

In [4]:
import urllib.request
import json
with open('table.zip', 'rb') as table, \
     open('chair.zip', 'rb') as chair, \
     open('bed.zip', 'rb') as bed:
        response = visrec.create_classifier(name="homeappliancesclassifier",
                                        positive_examples={'table': table, \
                                                           'chair': chair, \
                                                           'bed': bed})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "homeappliancesclassifier_1815988964",
  "name": "homeappliancesclassifier",
  "status": "training",
  "owner": "c4aa8115-d6d4-4e36-b8f6-aad0199ba066",
  "created": "2020-05-25T22:35:10.133Z",
  "updated": "2020-05-25T22:35:10.133Z",
  "classes": [
    {
      "class": "bed"
    },
    {
      "class": "chair"
    },
    {
      "class": "table"
    }
  ],
  "core_ml_enabled": true
}


In [5]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'homeappliancesclassifier_1815988964'

In [19]:
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

Good to go 


In [20]:
visrec.list_classifiers(verbose=True).get_result()

{'classifiers': [{'classifier_id': 'homeappliancesclassifier_1815988964',
   'name': 'homeappliancesclassifier',
   'status': 'ready',
   'owner': 'c4aa8115-d6d4-4e36-b8f6-aad0199ba066',
   'created': '2020-05-25T22:35:10.133Z',
   'updated': '2020-05-25T22:35:10.133Z',
   'classes': [{'class': 'bed'}, {'class': 'chair'}, {'class': 'table'}],
   'core_ml_enabled': True},
  {'classifier_id': 'fastfoodclassifier_1903261324',
   'name': 'fastfoodclassifier',
   'status': 'ready',
   'owner': 'c4aa8115-d6d4-4e36-b8f6-aad0199ba066',
   'created': '2020-05-23T10:34:06.760Z',
   'updated': '2020-05-23T10:34:06.760Z',
   'classes': [{'class': 'burger'}, {'class': 'pizza'}, {'class': 'fries'}],
   'core_ml_enabled': True}]}

In [21]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

In [33]:
getdf_visrec(url = 'https://cdn11.bigcommerce.com/s-1u1m3wn/images/stencil/1280x1280/products/2776/4074/Almore-light-grey-upholstered-queen-bed-with-buttoned-tufted-headboard__80916.1520481414.jpg?c=2&imbypass=on',
            classifier_ids=classifier_id)

,class,score
0,bed,0.907
